In [ ]:
import torch, os
import IPython.display
import numpy as np
import matplotlib.pyplot as plt
from src.models.model import VAE
from src.utils import *
from src.interact_func import *

In [ ]:
model = VAE(130, 2048, 512, 32)
model.eval()
dic = torch.load('params/tr_512.pt')
for name in list(dic.keys()):
    dic[name.replace('module.', '')] = dic.pop(name)
model.load_state_dict(dic)
if torch.cuda.is_available():
    model = model.cuda()



data1 = np.load('/scratch/dw1920/batch_0.npy')
# data2 = np.load('/scratch/dw1920/batch_1.npy')
# data3 = np.load('/scratch/dw1920/batch_2.npy')

In [ ]:
# data = torch.from_numpy(np.concatenate(np.array([data1, data2]), 0)).float().cuda()

all_z = []
batch_size = 1000
for i in range(5):
    data = torch.from_numpy(data1[i*batch_size:(i+1)*batch_size]).float().cuda()
    z = model.encode(data).mean
    all_z.append(z)
    print(i)


In [ ]:
type(all_z[0])

In [ ]:
from sklearn.decomposition import PCA

for i in range(len(all_z)):
    all_z[i] = all_z[i].cpu().detach().numpy()
x=np.concatenate(np.array(all_z), 0)
X_scaler = StandardScaler()
x = X_scaler.fit_transform(x)
pca = PCA(n_components=128)
pca.fit(x)
pca.transform(x)

In [ ]:
cov_mat = np.cov(x, rowvar = 0)

In [ ]:
sigma = cov_mat
[U,S,V] = np.linalg.svd(sigma)

In [ ]:
s_out = 0
for i in range(128):
    s_out += S[i]

t_out = 0
for i in range(512):
    t_out += S[i]
    
print(s_out/t_out)

In [ ]:
print(pca.explained_variance_ratio_)

In [ ]:
var = pca.explained_variance_ratio_
varz = {}

for i, j in enumerate(var):
    varz[i+1] = j


In [ ]:
sorted_by_value = sorted(varz.items(), key=lambda kv: kv[1]).reverse()


In [ ]:
sorted_by_value

In [ ]:
dims = [i[0] for i in sorted_by_value]
var_tmp = [i[1] for i in sorted_by_value]
var = np.cumsum(var_tmp)
plt.figure(figsize=(12, 10))
plt.ylabel('variances', fontsize=48)
plt.xlabel("dimension indexes", fontsize=48)
plt.plot(dims, var, color='red', linestyle='solid')
plt.title('')
plt.savefig('pca.png',bbox_inches="tight", dpi=300)
plt.show()
